<a href="https://colab.research.google.com/github/roshancharlie/College-Chatbot-Using-ML-and-NLP/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing The Neccesary Libraries

In [2]:
import nltk
import json
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random
import warnings
warnings.filterwarnings('ignore')

# Creating Query-Based Intents for the Dataset

In [3]:
intents = {
  "intents": [
    {
      "tag": "greeting",
      "patterns": ["Hi", "Hello", "Hey", "Good day", "How are you?"],
      "responses": ["Hello!", "Good to see you!", "Hi there, how can I help?"],
    },
    {
      "tag": "farewell",
      "patterns": ["Goodbye", "Bye", "See you later", "Talk to you later"],
      "responses": ["Sad to see you go :(", "Goodbye!", "Come back soon!"],

    },
    {
      "tag": "creator",
      "patterns": ["Who created you?", "Who is your developer?", "Who made you?"],
      "responses": ["I was created by Shivani & Arthiga"]

    },
    {
      "tag": "identity",
      "patterns": ["What is your name?", "What should I call you?", "Who are you?","What are you","Introduce Yourself"],
      "responses": ["You can call me Jarvis. I'm a Chatbot."]

    },
    
    {
      "tag": "casual_greeting",
      "patterns": ["What's up?", "How are you?", "How you doing?"],
       "responses": ["I'm here to assist you with any questions or information you need. How can I assist you today?"]

     },
    {
      "tag": "good_morning",
      "patterns": ["Good morning", "Morning"],
      "responses": ["Good morning! How can I assist you today?"]

     },
     {
       "tag": "good_afternoon",
       "patterns": ["Good afternoon", "Afternoon"],
        "responses": ["Good afternoon! How can I assist you today?"]

      },
      {
      "tag": "good_evening",
      "patterns": ["Good evening", "Evening"],
       "responses": ["Good evening! How can I assist you today?"]

         },
          {
        "tag": "thank_you",
        "patterns": ["Thank you", "Thanks"],
        "responses": ["You're welcome! If you have any more questions, feel free to ask."]

        },
       {
       "tag": "sorry",
      "patterns": ["Sorry", "Apologies"],
       "responses": ["No problem! If there's anything else you need assistance with, feel free to let me know."]

    },
    {
         "tag": "Total_Failures",
      "patterns": ["Total Failures","Today count of machine failure?","How many machines failed?"],
       "responses": ["Here are the results!"]
    },
    {
        "tag": "Total_Non_Failures",
      "patterns": ["Total Avalible machines","Today count of machine present?","How many machines available?"],
       "responses": ["Here are the results!"]
    }  ,
    { "tag": "Common_Failure_Conditions",
      "patterns": ["what are the conditions for failure","common failure conditions?"],
       "responses": ["Here are the results!"]
        
    },
    {
        "tag": "Average_Temperature",
      "patterns": ["what is the average temperature for failure","Average Temperature for failure"],
       "responses": ["Here are the results!"]
    },
    {
        "tag": "High_VOC",
      "patterns": ["what is the Failure Rate with High VOC "],
       "responses": ["Here are the results!"]
    },
    {
        "tag": "High_Footfall",
      "patterns": ["what is the Failure Rate with High Footfall? "],
       "responses": ["Here are the results!"]
    },
  {
       "tag": "CS_Level",
      "patterns": ["what is the Failure Rate with CS Level? "],
       "responses": ["Here are the results!"]
  },
  {
       "tag": "Poor_Air_Quality ",
      "patterns": ["what is the Failure Rate with Poor Air Quality ? "],
       "responses": ["Here are the results!"]
  },
  {
       "tag": "High_IP",
      "patterns": ["what is the Failure Rate with High IP? "],
       "responses": ["Here are the results!"]
  }
]
}

### Downloading NLP Package

In [4]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rockstar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rockstar\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Processing Data and Synonym Augmentation for Intent Classification

In [5]:
stemmer = WordNetLemmatizer()

words = []
classes = []
documents = []
ignore_words = ['?']


for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # Add to our words list
        words.extend(w)
        # Add to documents in our corpus
        documents.append((w, intent['tag']))
        # Add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])


words = [stemmer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


classes = sorted(list(set(classes)))

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique lemmatized words", words)

45 documents
19 classes ['Average_Temperature', 'CS_Level', 'Common_Failure_Conditions', 'High_Footfall', 'High_IP', 'High_VOC', 'Poor_Air_Quality ', 'Total_Failures', 'Total_Non_Failures', 'casual_greeting', 'creator', 'farewell', 'good_afternoon', 'good_evening', 'good_morning', 'greeting', 'identity', 'sorry', 'thank_you']
65 unique lemmatized words ["'s", 'afternoon', 'air', 'apology', 'are', 'available', 'avalible', 'average', 'bye', 'c', 'call', 'common', 'condition', 'count', 'created', 'day', 'developer', 'doing', 'evening', 'failed', 'failure', 'footfall', 'for', 'good', 'goodbye', 'hello', 'hey', 'hi', 'high', 'how', 'i', 'introduce', 'ip', 'is', 'later', 'level', 'machine', 'made', 'many', 'morning', 'name', 'of', 'poor', 'present', 'quality', 'rate', 'see', 'should', 'sorry', 'talk', 'temperature', 'thank', 'thanks', 'the', 'to', 'today', 'total', 'up', 'voc', 'what', 'who', 'with', 'you', 'your', 'yourself']


In [9]:
import numpy as np
import random

# Let's assume 'training' and 'augmented_data' are lists of [bag, output_row] pairs
training = []
output = []
output_empty = [0] * len(classes)

# Training set, bag of words for each sentence
for doc in documents:
    # Initialize our bag of words
    bag = []
    # List of tokenized words for the pattern
    pattern_words = doc[0]
    # Lemmatize each word
    pattern_words = [stemmer.lemmatize(word.lower()) for word in pattern_words]
    # Create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # Output is a '0' for each tag and '1' for the current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

random.shuffle(training)

def synonym_replacement(tokens, limit):
    augmented_sentences = []
    for i in range(len(tokens)):
        synonyms = []
        for syn in wordnet.synsets(tokens[i]):
            for lemma in syn.lemmas():
                synonyms.append(lemma.name())
        if len(synonyms) > 0:
            num_augmentations = min(limit, len(synonyms))
            sampled_synonyms = random.sample(synonyms, num_augmentations)
            for synonym in sampled_synonyms:
                augmented_tokens = tokens[:i] + [synonym] + tokens[i + 1:]
                augmented_sentences.append(' '.join(augmented_tokens))
    return augmented_sentences

# Augment the training data using synonym replacement
augmented_data = []
limit_per_tag = 100

for i, doc in enumerate(training):
    bag, output_row = doc
    tokens = [words[j] for j in range(len(words)) if bag[j] == 1]
    augmented_sentences = synonym_replacement(tokens, limit_per_tag)
    for augmented_sentence in augmented_sentences:
        augmented_bag = [1 if word in augmented_sentence.split() else 0 for word in words]
        augmented_data.append([augmented_bag, output_row])

# Check the structure of the data and ensure consistency
print("Length of training data:", len(training))
print("Length of augmented data:", len(augmented_data))

# Combine training and augmented data (no need for numpy arrays if the structure is inconsistent)
combined_data = training + augmented_data
random.shuffle(combined_data)

# Optional: Convert to numpy array if shapes are consistent, but this should work as is
# combined_data = np.array(combined_data, dtype=object)


Length of training data: 45
Length of augmented data: 2153


# Splitting the Dataset

In [10]:
from sklearn.model_selection import train_test_split


def separate_data_by_tags(data):
    data_by_tags = {}
    for d in data:
        tag = tuple(d[1])
        if tag not in data_by_tags:
            data_by_tags[tag] = []
        data_by_tags[tag].append(d)
    return data_by_tags.values()


separated_data = separate_data_by_tags(combined_data)

# Lists to store training and testing data
training_data = []
testing_data = []

# Split each tag's data into training and testing sets
for tag_data in separated_data:
    train_data, test_data = train_test_split(tag_data, test_size=0.2, random_state=42)
    training_data.extend(train_data)
    testing_data.extend(test_data)


random.shuffle(training_data)
random.shuffle(testing_data)

# Convert training and testing data back to np.array
train_x = np.array([d[0] for d in training_data])
train_y = np.array([d[1] for d in training_data])
test_x = np.array([d[0] for d in testing_data])
test_y = np.array([d[1] for d in testing_data])

# Training The Model

In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.dropout1 = nn.Dropout(0.2)

        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm1d(hidden_size)
        self.dropout2 = nn.Dropout(0.2)

        self.fc3 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.bn1(x)
        x = self.dropout1(x)

        x = self.fc2(x)
        x = self.relu2(x)
        x = self.bn2(x)
        x = self.dropout2(x)

        x = self.fc3(x)
        output = self.softmax(x)
        return output

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return self.softmax(x)

class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def accuracy(predictions, targets):
    predicted_labels = torch.argmax(predictions, dim=1)
    true_labels = torch.argmax(targets, dim=1)
    correct = (predicted_labels == true_labels).sum().item()
    total = targets.size(0)
    return correct / total

def test_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_accuracy = 0.0
    num_batches = len(test_loader)

    with torch.no_grad():
        for inputs, targets in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * inputs.size(0)
            total_accuracy += accuracy(outputs, targets) * inputs.size(0)

    average_loss = total_loss / len(test_loader.dataset)
    average_accuracy = total_accuracy / len(test_loader.dataset)
    return average_loss, average_accuracy

In [12]:
# Create DataLoader for training and testing data
train_x = torch.tensor(train_x).float()
train_y = torch.tensor(train_y).float()
test_x = torch.tensor(test_x).float()
test_y = torch.tensor(test_y).float()

batch_size = 64
train_dataset = CustomDataset(train_x, train_y)
test_dataset = CustomDataset(test_x, test_y)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the model, loss function, and optimizer
input_size = len(train_x[0])
hidden_size = 8
output_size = len(train_y[0])
model = NeuralNetwork(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters())

# Train the model and evaluate on the testing set
num_epochs = 50
for epoch in range(num_epochs):
    # Training
    model.train()
    running_loss = 0.0
    running_acc = 0.0
    for inputs, targets in train_loader:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, targets)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Update statistics
        running_loss += loss.item() * inputs.size(0)
        running_acc += accuracy(outputs, targets) * inputs.size(0)

    # Calculate average training loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader.dataset)
    epoch_acc = running_acc / len(train_loader.dataset)

    # Print training loss and accuracy for each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_acc:.4f}")

    # Evaluate on the testing set
    test_loss, test_accuracy = test_model(model, test_loader, criterion)
    print(f"Epoch [{epoch+1}/{num_epochs}], Testing Loss: {test_loss:.4f}, Testing Accuracy: {test_accuracy:.4f}")

# Save the trained model
torch.save(model.state_dict(), 'model.pth')

Epoch [1/50], Training Loss: 0.2076, Training Accuracy: 0.0400
Epoch [1/50], Testing Loss: 0.2068, Testing Accuracy: 0.0404
Epoch [2/50], Training Loss: 0.2062, Training Accuracy: 0.0400
Epoch [2/50], Testing Loss: 0.2054, Testing Accuracy: 0.0404
Epoch [3/50], Training Loss: 0.2045, Training Accuracy: 0.0405
Epoch [3/50], Testing Loss: 0.2035, Testing Accuracy: 0.0404
Epoch [4/50], Training Loss: 0.2023, Training Accuracy: 0.1147
Epoch [4/50], Testing Loss: 0.2008, Testing Accuracy: 0.1525
Epoch [5/50], Training Loss: 0.1989, Training Accuracy: 0.1849
Epoch [5/50], Testing Loss: 0.1967, Testing Accuracy: 0.1816
Epoch [6/50], Training Loss: 0.1940, Training Accuracy: 0.2003
Epoch [6/50], Testing Loss: 0.1910, Testing Accuracy: 0.2018
Epoch [7/50], Training Loss: 0.1877, Training Accuracy: 0.2209
Epoch [7/50], Testing Loss: 0.1842, Testing Accuracy: 0.2197
Epoch [8/50], Training Loss: 0.1809, Training Accuracy: 0.2317
Epoch [8/50], Testing Loss: 0.1775, Testing Accuracy: 0.2309
Epoch [9

# Model Inference

In [13]:
def load_model(model_path, input_size, hidden_size, output_size):
    model = NeuralNetwork(input_size, hidden_size, output_size)
    model.load_state_dict(torch.load(model_path))
    model.eval()
    return model

# Function to preprocess the input sentence
def preprocess_sentence(sentence, words):
    sentence_words = sentence.lower().split()
    sentence_words = [word for word in sentence_words if word in words]
    return sentence_words

# Function to convert the preprocessed sentence into a feature vector
def sentence_to_features(sentence_words, words):
    features = [1 if word in sentence_words else 0 for word in words]
    return torch.tensor(features).float().unsqueeze(0)

# Function to generate a response using the trained model
def generate_response(sentence, model, words, classes):
    sentence_words = preprocess_sentence(sentence, words)
    if len(sentence_words) == 0:
        return "I'm not quite sure I get it. Could you explain that a bit more or put it another way?"

    features = sentence_to_features(sentence_words, words)
    with torch.no_grad():
        outputs = model(features)

    probabilities, predicted_class = torch.max(outputs, dim=1)
    confidence = probabilities.item()
    predicted_tag = classes[predicted_class.item()]

    if confidence > 0.5:
        for intent in intents['intents']:
            if intent['tag'] == predicted_tag:
                return random.choice(intent['responses'])

    return "I'm a bit lost on how to respond. Could you explain a bit more?"

In [14]:
model_path = 'model.pth'
input_size = len(words)
hidden_size = 8
output_size = len(classes)
model = load_model(model_path, input_size, hidden_size, output_size)

# Test the chatbot response
print('Hello! This is JARVIS. How can I assist you today? Type "quit" to exit.')
# while True:
#     user_input = input('> ')
#     if user_input.lower() == 'quit':
#         break
#     response = generate_response(user_input, model, words, classes)
#     print(response)

Hello! This is JARVIS. How can I assist you today? Type "quit" to exit.
I'm not quite sure I get it. Could you explain that a bit more or put it another way?
I'm not quite sure I get it. Could you explain that a bit more or put it another way?


: 

In [1]:
print("jarvis")

jarvis
